In [1]:
%run -i PFAcode.ipynb

In [ ]:
#STATS (binary strings only)
print(len(SC))
for l in range(3,11):
    if l<9:
        top=4;
    else:
        top=5;
    for c in range(1,top):
        print("len "+str(l)+" complexity "+str(c)+": "+str(lookup(SC,c,l).cardinality())+" (NFA: "+str(lookup(NC,c,l).cardinality())+")")
    nu = lookup(SC,100,l,less=True).cardinality();
    if nu < 2^l:
        print("total classified "+str(nu)+"/"+str(2^l));
    witsum = sum([len(SW[k]) for k in SW.keys() if len(k)==l]);
    print("total witnesses "+str(witsum));
    print("");
print("# strings of each length with PFA=NFA complexity:");
for l in range(3,11):
    print(str(l)+": "+str(len([k for k in SC.keys() if len(k) == l and SC[k] == NC[k]])))
print("# strings of each length with PFA=NFA complexity + 1:")
for l in range(3,11):
    print(str(l)+": "+str(len([k for k in SC.keys() if len(k) == l and SC[k] == NC[k] + 1])))

In [3]:
test2x2 = bruteforce(2,2);
test2x3 = bruteforce(2,3);
test2x4 = bruteforce(2,4);
test2x5 = bruteforce(2,5);
test2x6 = bruteforce(2,6);
test2x7 = bruteforce(2,7);
test2x8 = bruteforce(2,8);
test3x2 = bruteforce(3,2);
half3x3 = bruteforce(3,3,permute=False);

In [33]:
three2x2 = bruteforce(2,2,range(3));
three2x3 = bruteforce(2,3,range(3));
three2x4 = bruteforce(2,4,range(3));
three2x5 = bruteforce(2,5,range(3));
three2x6 = bruteforce(2,6,range(3));
three2x8 = bruteforce(2,8,range(3));

In [6]:
somevecs = [ [1,0,0],[0,1,0],[0,0,1],[1/4,1/4,1/2],
           [1/4,1/2,1/4],[1/2,1/4,1/4]];
some3x4 = bruteforcevecs(maketuple(somevecs),permute=True);

somevecs2 = [ [1,0,0],[0,1,0],[0,0,1], [1/4,0,3/4],
             [1/4,3/4,0], [3/4,0,1/4], [3/4,1/4,0],
             [0,1/4,3/4], [0,3/4,1/4]];
some3x4_2 = bruteforcevecs(maketuple(somevecs2),permute=False);

In [7]:
print(len(test3x2))
print(len(half3x3))
print(len(some3x4))
print(len(some3x4_2))

46656
499500
46656
265356


In [3]:
#given a list of strings (as tuples/lists), a pool of matrix dicts, and start/accept states,
#try to find a witness for each string in the pool.
#will print any successes, and stop after trying maxattempts strings. If maxattemps=0, try all of them.
#won't attempt to find witnesses if one already exists.
def tryfindwitness(strings,pool,start_states=None,accept_states=None,maxattempts=50):
    c = 0;
    for s in strings:
        if (s in SW.keys() and len(SW[s]) > 0) or NC[s] == 1:
            continue;
        c = c + 1;
        if c < 0:
            continue;
        if c > maxattempts and maxattempts != 0:
            break;
        #sys.stdout.write("%d\r" % (c) );
        print(c,s);
        v = start_states;
        f = accept_states;
        wits = findwitness(pool,s,v,f);
        if wits != None:
            print(s,wits,addonewit(wits,s,v,f));
            print("");
            print(flip(s),switchmats(wits),addonewit(switchmats(wits),flip(s),v,f));
            print("");
        else:
            print("no witness found");
            print("");

In [20]:
savewits()

In [4]:
rand10 = [s for s in NC.keys() if len(s)==10 and isNFArand(s)];
print(len(rand10))
rand11 = [s for s in NC.keys() if len(s)==11 and isNFArand(s)];
print(len(rand11))
rand12 = [s for s in NC.keys() if len(s)>=12 and isNFArand(s)];
print(len(rand12))

344
449
390


In [9]:
pool2_1 = test2x2+test2x3+test2x4+test2x5+test2x6
print(len(pool2_1))
pool2_2 = test2x8+bruteforce(2,10)
print(len(pool2_2))

4659
21202


In [35]:
test2x20 = bruteforce(2,20)
print(len(test2x20))

194481


In [10]:
pool3_1 = test3x2+half3x3
print(len(pool3_1))

546156


In [ ]:
tryfindwitness(rand12,pool3_1,[1,0,0],[0,0,1],maxattempts=10)

In [ ]:
tryfindwitness(rand12,pool3_1,[1,0,0],[1,0,0],maxattempts=10)

In [53]:
savewits()

In [20]:
#https://gist.github.com/the-moog/94b09b49232731bd2a3cedd24501e23b
import threading
from IPython.display import display
import ipywidgets as widgets
import time

def get_ioloop():
    import IPython, zmq
    ipython = IPython.get_ipython()
    if ipython and hasattr(ipython, 'kernel'):
        return zmq.eventloop.ioloop.IOLoop.instance()


#The IOloop is shared
ioloop = get_ioloop()


class MyThread(threading.Thread):
    def __init__(self):
        super().__init__()
        
        self.progress = widgets.FloatProgress(value=0, min=0, max=10)
        
        self._quit = threading.Event()
        self.start()
    
    def run(self):
        i = 0
        while not self._quit.isSet():
            def update_progress(i=i):
                if self._quit.isSet():
                    return
                if self.progress.value == 10:
                    self.progress.value = 0
                self.progress.value = i

            time.sleep(2)

            ioloop.add_callback(update_progress)

            i += 1
            if i > 10:
                i = 0
        self.progress.value = 10
        print("Quit")
            
    def quit(self):
        self._quit.set()

def run_progress():
    thread = MyThread()
    display(thread.progress)
    return thread

In [21]:
t1 = run_progress()

FloatProgress(value=0.0, max=10.0)

In [27]:
t1.quit()

Quit


In [ ]:
proc = multiprocessing.Process(target=tryfindwitness,args=(rand10,pool2_1,[1,0],[0,1],maxattempts=10));
proc.start()

In [146]:
#really inefficient (for larger sets of strings)
c = 0;
for s in rand5:
    #if len(SW[s]) > 0:
    #    continue;
    c = c + 1;
    if c < 0:
        continue;
    if c > 50:
        break;
    sys.stdout.write("%d\r" % (c) );
    v = [1,0];
    f = [0,1];
    #f = list(reversed(f));
    wits = findwitness(test4state[::],s,v,f);
    #wits = findwitness(half3x3[269502::91],s,v,f);
    #wits = findwitness(test2x15[::13],s,v,f);
    if wits != None:
        print(s,wits,addonewit(wits,s,v,f));
        print("");
        print(flip(s),switchmats(wits),addonewit(switchmats(wits),flip(s),v,f));
        print("");

In [ ]:
[(k,NC[k]) for k in NC.keys() if isNFArand(k) and len(k) < 5]

In [163]:
strs = [k for k in NC.keys() if isNFArand(k) and len(k) < 6];

In [5]:
#test if s is a substring of t. Expects them as iterables!
def testsubstr(s,t):
    if len(s) >= len(t) and s != t: return False;
    is_substr = False;
    for ind in range(len(t)-len(s)+1):
        if s == t[ind:ind+len(s)]:
            is_substr = True;
    return is_substr;

In [ ]:
for s in strs:
    for t in strs:
        #if s == t or s == tuple(reversed(t)) or s == tuple(flip(t)): continue;
        #avoid strings of same length, and duplication
        if len(s) >= len(t): continue
        if testsubstr(s,t): continue
        if testsubstr(s,tuple(reversed(t))): continue
        if testsubstr(s,tuple(flip(t))): continue
        if testsubstr(s,tuple(flip(reversed(t)))): continue
        #I KNOW there has to be a better way to do this
        con = []
        con.append([s+t,"s+t"])
        con.append([t+s,"t+s"])
        con.append([s+tuple(reversed(t)),"s+rev(t)"])
        con.append([s+tuple(flip(t)),"s+fl(t)"])
        con.append([s+tuple(flip(reversed(t))),"s+flrev(t)"])
        con.append([t+tuple(reversed(s)),"t+rev(s)"])
        con.append([t+tuple(flip(s)),"t+fl(s)"])
        con.append([t+tuple(flip(reversed(s))),"t+flrev(s)"])
        testrand = []
        for i in range(len(con)):
            teststr = con[i][0]
            if not (teststr in NC.keys()):
                #print(teststr)
                NC[teststr] = nfacomplexity(teststr)
                pickletofile(NC,"nfa_complexity")
                sleep(1)
            testrand.append(isNFArand(teststr))
        value = False #disjunction of all truth values - want to see if they all fail
        for i in range(len(testrand)):
            value = value or i
        #continue if any of them succeeded
        #if value: continue
        print(s,t)
        for i in range(len(con)):
            print(con[i],NC[con[i][0]])
        print("")

In [ ]:
#fully distinct strings of length 5 with A_P=3:
rand5 = [k for k in S2[5] if k[0]==0 and SC[k]==3]; rand5

In [45]:
test2x25 = bruteforce(2,25);
print(len(test2x25))

456976


In [49]:
uniq2x20 = finduniques(test2x20,strlen=5,start_states=[1,0],accept_states=[0,1]);

In [50]:
len(uniq2x20)

170548

In [6]:
def tracegaps(string,autlist,start=(1,0),accept=(0,1)):
    gap = QQ(1);
#FIXME: replace 2 with size of alphabet
    numhigher = 2^len(string) - 1;
    c = 0;
    for aut in autlist:
        c = c+1;
        sys.stdout.write("%d\r" % (c) );
        thelen = ZZ(len(string));
        thelist = sortproblist(list_probs(aut,start_states=start,accept_states=accept,lengths=thelen));
        highstr = highest_prob(thelist);
        #get index within thelist of string
        thelist_juststr = [s[0] for s in thelist];
        highstr_juststr = [s[0] for s in highstr];
        theind = thelist_juststr.index(string);
        #if string is among those with highest probability, print that info but
        #don't update the gap or index, because it could be a rather indiscriminate automaton.
        if string in highstr_juststr:
            if len(highstr_juststr) == 1:
                print("c = "+str(c));
                print("WITNESS FOUND",highstr,aut);
                print("");
            elif 1 < len(highstr_juststr) < numhigher: #exclude case where lots of strings are accepted with same prob
                print("c = "+str(c));
                print(str(string)+" among highest-prob strings: "+str(highstr),aut);
                print("");
        #logic: if theind < numhigher, only then decide whether to update the gap
        #if theind < numhigher and the gap is bigger, print that gap without updating it
        else:
            acc = accprob(aut,start_states=start,accept_states=accept,sigma=string);
            if acc == 0: continue;
            thisgap = highstr[0][1] - acc;
            if theind <= numhigher or numhigher == 0:
                if thisgap <=0: #this code should never execute
                    print("c = "+str(c));
                    print("Gap of "+str(thisgap)+" with index "+str(theind)+", not updating",aut);
                    print("");
                elif thisgap < gap and theind <= numhigher:
                    print("c = "+str(c));
                    gap = thisgap;
                    numhigher = theind;
                    print("Gap of "+str(gap)+" and non-increased index "+str(theind),aut);
                    print(thelist[0:theind+2]);
                    print("");
                elif thisgap >= gap and theind < numhigher:
                    print("c = "+str(c));
                    gap = thisgap; #experimental: tolerate increasing the gap if we get closer to the top
                    numhigher = theind;
                    print("Larger gap of "+str(thisgap)+", but new index "+str(theind),aut);
                    print(thelist[0:theind+2]);
                    print("");
    return;

In [86]:
test2state = bruteforcevecs(( #(0.99,0.01),(0.01,0.99),
                             (0.54,0.46),(0.46,0.54),
                             (0.55,0.45),(0.45,0.55),
                             (0.49,0.51),(0.51,0.49),
                             (0.89,0.11),(0.11,0.89),(0.88,0.12),(0.12,0.88),(0.545,0.455),(0.455,0.545),
                             (0.44,0.56),(0.56,0.44),(0.5425,1-0.5425),(1-0.5425,0.5425)),
                           makerational=True);
len(test2state)

65536

In [8]:
test3state = bruteforceprobs(2, [0.49,0.11,0.44,0.455,0.4475,0.45,0.4575,0.455,0.4525,0.46]);
len(test3state)

160000

In [ ]:
tracegaps( (0,1,1,0,1), test3state[21500:], (1,0),(0,1))

In [ ]:
tracegaps( (0,0,1,0,0), test3state, (1,0),(1,0))

In [102]:
tracegaps( (0,1,1,0,1), test2x25, (1,0),(0,1))

Gap of 3906/1953125 and new index 2 {0: [24/25  1/25]
[ 9/25 16/25], 1: [22/25  3/25]
[    1     0]}
[((0, 0, 0, 0, 1), 8559/78125), ((1, 1, 0, 0, 1), 42093/390625), ((0, 1, 1, 0, 1), 210069/1953125), ((1, 0, 1, 0, 1), 42003/390625), ((1, 0, 0, 0, 1), 8397/78125), ((1, 1, 1, 0, 1), 209907/1953125), ((0, 1, 0, 0, 1), 41931/390625), ((0, 0, 1, 1, 1), 209409/1953125), ((0, 1, 0, 1, 1), 209319/1953125), ((1, 1, 1, 1, 1), 1046343/9765625), ((1, 0, 0, 1, 1), 41853/390625), ((1, 0, 1, 1, 1), 209247/1953125), ((0, 1, 1, 1, 1), 1046181/9765625), ((0, 0, 1, 0, 1), 41841/390625), ((1, 1, 0, 1, 1), 209157/1953125), ((0, 0, 0, 1, 1), 41691/390625), ((0, 0, 0, 1, 0), 8309/78125), ((1, 1, 0, 1, 0), 40843/390625), ((0, 0, 1, 0, 0), 8159/78125), ((0, 1, 1, 1, 0), 203819/1953125), ((1, 0, 1, 1, 0), 40753/390625), ((1, 0, 0, 1, 0), 8147/78125), ((1, 1, 1, 1, 0), 203657/1953125), ((0, 1, 0, 1, 0), 40681/390625), ((0, 0, 1, 1, 0), 40591/390625), ((0, 1, 0, 0, 0), 8069/78125), ((1, 1, 1, 0, 0), 40093/390625

In [ ]:
tracegaps( (0,0,1,0,0), test2x20, (1,0),(0,1))

In [145]:
test4state = bruteforceprobs(2, [0.01,0.49,0.499,0.48,0.2,0.44,0.455,0.4475,0.45,0.4575,0.455,0.4525,0.46]);
len(test4state)

456976

In [5]:
probs5 = [0.01,0.001];
new1 = 0.430;
while new1 <= 0.460:
    probs5.append(new1);
    new1 = new1 + 0.005;
new2 = 0.490;
while new2 < 0.500:
    probs5.append(new2);
    new2 = new2 + 0.001;
test5state = bruteforceprobs(2,probs5);
print(len(test5state))

2085136


In [33]:
tracegaps( (0,1,1,0,1), test5state[200000:], (1,0),(0,1) )

c = 71
Gap of 616623/19531250 and non-increased index 4 {0: [99/100  1/100]
[43/100 57/100], 1: [ 89/200 111/200]
[113/200  87/200]}
[((0, 0, 0, 0, 1), 43167261/78125000), ((1, 0, 0, 0, 1), 42269973/78125000), ((1, 1, 0, 0, 1), 41777229/78125000), ((0, 1, 0, 0, 1), 41584953/78125000), ((0, 1, 1, 0, 1), 40700769/78125000), ((1, 1, 1, 0, 1), 40659567/78125000)]

c = 109
Gap of 125185023/4000000000 and non-increased index 4 {0: [ 99/100   1/100]
[ 87/200 113/200], 1: [ 89/200 111/200]
[113/200  87/200]}
[((0, 0, 0, 0, 1), 11051184507/20000000000), ((1, 0, 0, 0, 1), 21655153677/40000000000), ((1, 1, 0, 0, 1), 1337703177/2500000000), ((0, 1, 0, 0, 1), 2663319453/5000000000), ((0, 1, 1, 0, 1), 81447339/156250000), ((1, 1, 1, 0, 1), 325462683/625000000)]

c = 147
Gap of 15510627/500000000 and non-increased index 4 {0: [99/100  1/100]
[ 11/25  14/25], 1: [ 89/200 111/200]
[113/200  87/200]}
[((0, 0, 0, 0, 1), 11051547/20000000), ((1, 0, 0, 0, 1), 21667857/40000000), ((1, 1, 0, 0, 1), 13385277/

In [52]:
probs6 = [];
new1 = 0.450;
while new1 < 0.460:
    probs6.append(new1);
    new1 = new1 + 0.002;
new2 = 0.490;
while new2 <= 0.500:
    probs6.append(new2);
    new2 = new2 + 0.0005;
test6state = bruteforceprobs(2,probs6,info=True);
print(len(test6state))

51 possible entries
51 rows
2601 matrices
6765201 matrix tuples
6765201


In [53]:
tracegaps( (0,1,1,0,1), test6state, (1,0),(0,1) )

c = 2
Gap of 11/10000 and non-increased index 22 {0: [137/250 113/250]
[  11/20    9/20], 1: [11/20  9/20]
[11/20  9/20]}
[((0, 0, 0, 1, 0), 4511/10000), ((0, 0, 1, 1, 0), 4511/10000), ((0, 1, 0, 1, 0), 4511/10000), ((0, 1, 1, 1, 0), 4511/10000), ((1, 0, 0, 1, 0), 4511/10000), ((1, 0, 1, 1, 0), 4511/10000), ((1, 1, 0, 1, 0), 4511/10000), ((1, 1, 1, 1, 0), 4511/10000), ((0, 1, 0, 0, 0), 1127744511/2500000000), ((1, 1, 0, 0, 0), 1127744511/2500000000), ((0, 0, 0, 0, 0), 7048403193613/15625000000000), ((1, 0, 0, 0, 0), 563872255489/1250000000000), ((0, 0, 1, 0, 0), 2255489/5000000), ((0, 1, 1, 0, 0), 2255489/5000000), ((1, 0, 1, 0, 0), 2255489/5000000), ((1, 1, 1, 0, 0), 2255489/5000000), ((0, 0, 0, 0, 1), 9/20), ((0, 0, 0, 1, 1), 9/20), ((0, 0, 1, 0, 1), 9/20), ((0, 0, 1, 1, 1), 9/20), ((0, 1, 0, 0, 1), 9/20), ((0, 1, 0, 1, 1), 9/20), ((0, 1, 1, 0, 1), 9/20), ((0, 1, 1, 1, 1), 9/20)]

c = 52
Gap of 1127254509/1250000000000 and non-increased index 22 {0: [  11/20    9/20]
[137/250 113/250

KeyboardInterrupt: 

In [67]:
lookup(NC,5,8).intersection(lookup(SC,2,8))

{(0, 0, 0, 0, 1, 1, 1, 1), (1, 1, 1, 1, 0, 1, 0, 1), (0, 0, 0, 0, 1, 0, 1, 0), (0, 0, 0, 0, 1, 1, 1, 0), (1, 1, 1, 1, 0, 0, 0, 0), (1, 1, 1, 1, 0, 0, 0, 1)}

In [114]:
thisx = (0, 0, 0, 0, 1, 0, 1, 0)
mingap = 0.001
v = (1,0)
f = (1,0)
f = list(reversed(f))
thiswits = findwitness(test2x8,thisx,v,f,stop=0)
for w in thiswits:
    slist = sortproblist(list_probs_fromwits(w,lengths=Integer(len(thisx))))
    thisgap = N(slist[0][1] - slist[1][1])
    if thisgap > mingap:
        print(w,N(slist[0][1]),"gap",thisgap)
        print("")

({0: [1/8 7/8]
[5/8 3/8], 1: [  0   1]
[3/4 1/4]}, [1 0], [0]
[1]) 0.592529296875000 gap 0.00109863281250000



In [ ]:
SW[thisx]

In [92]:
savewits()

In [ ]:
[s for s in SC.keys() if SC[s] >= floor(len(s)/2)+1]

In [10]:
some3x5 = bruteforceprobs(3,(0,1/5),permute=False,info=True)

4 possible entries
9 rows
729 matrices
265356 matrix tuples


In [17]:
for s in NC.keys():
    if len(s) > 6: continue;
    if not isNFArand(s): continue;
    test = False;
    if isNFArand(s+(0,)) or isNFArand(s+(1,)): test = True;
    if test == False:
        print(s,NC[s])
        print(s+(0,),NC[s+(0,)])
        print(s+(1,),NC[s+(1,)])
        print("")

(0, 0, 1, 1, 1) 3
(0, 0, 1, 1, 1, 0) 3
(0, 0, 1, 1, 1, 1) 3

(1, 1, 0, 0, 0) 3
(1, 1, 0, 0, 0, 0) 3
(1, 1, 0, 0, 0, 1) 3



In [22]:
SW[(0,1,1,0)]

[(
{0: [     1/100     99/100]                            
[5001/10000 4999/10000], 1: [51/100 49/100]         [1]
[ 1/100 99/100]}                           , [1 0], [0]
),
 (
{0: [  0   1]                      
[1/2 1/2], 1: [1/2 1/2]         [1]
[  0   1]}             , [1 0], [0]
),
 (
{0: [5/8 3/8]                      
[  1   0], 1: [3/4 1/4]         [1]
[1/2 1/2]}             , [1 0], [0]
)]

In [25]:
for w in SW[(0,1,1,0)]:
    slist = sortproblist(list_probs_fromwits(w,lengths=4))
    thisgap = N(slist[0][1] - slist[1][1])
    print(w,N(slist[0][1]),"gap",thisgap)
    print("")

({0: [     1/100     99/100]
[5001/10000 4999/10000], 1: [51/100 49/100]
[ 1/100 99/100]}, [1 0], [1]
[0]) 0.491523250000000 gap 0.0576232624500000

({0: [  0   1]
[1/2 1/2], 1: [1/2 1/2]
[  0   1]}, [1 0], [1]
[0]) 0.500000000000000 gap 0.0625000000000000

({0: [5/8 3/8]
[  1   0], 1: [3/4 1/4]
[1/2 1/2]}, [1 0], [1]
[0]) 0.750976562500000 gap 0.00292968750000000



In [ ]:
for w in SW[(0,1,1,0)]:
    slist = sortproblist(list_probs_fromwits(w,lengths=5))
    print(w,slist)
    print("")

In [129]:
v = (1,0)
f = (0,1)
f=tuple(reversed(f))
uniq3 = finduniques(three2x2,7,v,f);
print(len(uniq3))

141


In [130]:
selectlist = [u for u in uniq3 if usesletters(u[1][0],3)];
print(len(selectlist))

6


In [132]:
addwitsnew(selectlist,v,f)

12

In [133]:
savewits()

In [8]:
print(len(S3[4].difference(Set(SC.keys()))))

6


In [7]:
sig = (0,1,1,0,1)
witsig = SW[(0,1,1,0,1)][0]
print(witsig)

({0: [1/2 1/2   0]
[  1   0   0]
[  1   0   0], 1: [1/2   0 1/2]
[  1   0   0]
[1/2 1/2   0]}, [1 0 0], [0]
[0]
[1])


In [154]:
pathlist(SW[(0,1,1,0,1)][0][0],(0,1,1,0,1),(0,0,1))

path [0, 0, 0, 0, 0, 2] has prob 1/32
path [0, 1, 0, 0, 0, 2] has prob 1/16
path [0, 0, 2, 0, 0, 2] has prob 1/32
path [0, 0, 2, 1, 0, 2] has prob 1/16
path [0, 0, 0, 2, 0, 2] has prob 1/16
path [0, 1, 0, 2, 0, 2] has prob 1/8


In [153]:
pathlist(SW[(0,1,1,0,1)][0][0],(0,0,0,0,1),(0,0,1))

path [0, 0, 0, 0, 0, 2] has prob 1/32
path [0, 1, 0, 0, 0, 2] has prob 1/16
path [0, 0, 1, 0, 0, 2] has prob 1/16
path [0, 0, 0, 1, 0, 2] has prob 1/16
path [0, 1, 0, 1, 0, 2] has prob 1/8


In [149]:
pathlist(SW[(0,1,1,0,1)][0][0],(1,1,0,1,0),(0,0,1))

In [151]:
sortproblist(list_probs_fromwits(SW[(0,1,1,0,1)][0],lengths=5))

[((0, 1, 1, 0, 1), 3/8),
 ((0, 0, 0, 0, 1), 11/32),
 ((0, 0, 0, 1, 1), 11/32),
 ((0, 0, 1, 0, 1), 11/32),
 ((0, 1, 0, 0, 1), 11/32),
 ((0, 1, 0, 1, 1), 11/32),
 ((1, 0, 0, 0, 1), 11/32),
 ((1, 0, 0, 1, 1), 11/32),
 ((1, 0, 1, 0, 1), 11/32),
 ((1, 1, 1, 0, 1), 11/32),
 ((1, 1, 0, 0, 1), 5/16),
 ((1, 1, 0, 1, 1), 5/16),
 ((0, 1, 1, 1, 1), 9/32),
 ((1, 1, 1, 1, 1), 9/32),
 ((0, 0, 1, 1, 1), 1/4),
 ((1, 0, 1, 1, 1), 1/4),
 ((0, 0, 0, 0, 0), 0),
 ((0, 0, 0, 1, 0), 0),
 ((0, 0, 1, 0, 0), 0),
 ((0, 0, 1, 1, 0), 0),
 ((0, 1, 0, 0, 0), 0),
 ((0, 1, 0, 1, 0), 0),
 ((0, 1, 1, 0, 0), 0),
 ((0, 1, 1, 1, 0), 0),
 ((1, 0, 0, 0, 0), 0),
 ((1, 0, 0, 1, 0), 0),
 ((1, 0, 1, 0, 0), 0),
 ((1, 0, 1, 1, 0), 0),
 ((1, 1, 0, 0, 0), 0),
 ((1, 1, 0, 1, 0), 0),
 ((1, 1, 1, 0, 0), 0),
 ((1, 1, 1, 1, 0), 0)]

In [6]:
a,b,c,d,e,f,g,h,i=var('a,b,c,d,e,f,g,h,i')

In [157]:
M = Matrix( [[ a,b,c,(a+b)/2,(a+b)/2,c],
             [d,e,f,d,e,f],
             [g,h,i,g,h,i]] )

In [167]:
Matrix(QQ, [[1/16,5/16,5/8, 1/4,1/4,1/2],
            [0,1,0,0,1,0],
            [0,0,1,0,0,1]]).rref()

[ 1  0  0  4 -1 -2]
[ 0  1  0  0  1  0]
[ 0  0  1  0  0  1]

In [175]:
Matrix([[a,b,c,a,c,b],
            [0,1,0,0,1,0],
            [0,0,1,0,0,1]]).rref()

[         1          0          0          1 -b/a + c/a  b/a - c/a]
[         0          1          0          0          1          0]
[         0          0          1          0          0          1]

In [ ]:
Matrix( [[1/16,5/16,5/8],[0,1,0],[0,0,1]]) * Matrix([[4,-1,-2],[0,1,0],[0,0,1]]) * Matrix( [[1/12,1/3,7/12],[0,1,0],[0,0,1]])

[ 1/48   1/3 31/48]
[    0     1     0]
[    0     0     1]

In [176]:
Matrix([0,0,1]).transpose()*Matrix([1,0,0])

[0 0 0]
[0 0 0]
[1 0 0]

In [84]:
#returns an m-adic PFA, a la Salomaa/Turakainen. It is a tuple: (matrices, initial, accepting) (as in SW)
#input: homo, a dict giving a homomorphism. Keys are generators of the language, values are their images (all strings)
def madic(homo):
    alph = list(homo.keys());
    m = max([int(a) for a in alph])+1;
    themats = {}; #matrix dict to generate
    for letter in alph:
        phi = madic_expansion(homo[letter],m);
        malpha = m^(-len(homo[letter]));
        themats.update({letter: Matrix(QQ, [[malpha, 1-malpha-phi, phi],
                                           [0,1,0],
                                           [0,0,1]])});
    return (themats, Matrix([1,0,0]), Matrix([0,0,1]).transpose());

#given a PFA in "witness" form, return a new PFA whose transition matrices are multiplied by the matrices
#in mask. That is, matrices[letter] becomes matrices[letter]*mask[letter]. (Right multiplication by default,
#but set left=True if you want left multiplication.)
#Does NOT check the types of anything or the sizes of matrices
def multiplyPFA(thePFA,mask,left=False):
    newmats = {};
    for letter in thePFA[0].keys():
        assert letter in mask.keys();
        if left:
            newmats.update({letter:mask[letter]*thePFA[0][letter]});
        else:
            newmats.update({letter:thePFA[0][letter]*mask[letter]});
    return (newmats,thePFA[1],thePFA[2]);

In [85]:
testhom = {'0':'0', '1':'1'};
testmad = madic(testhom)
print(testmad)

({'0': [1/2 1/2   0]
[  0   1   0]
[  0   0   1], '1': [1/2   0 1/2]
[  0   1   0]
[  0   0   1]}, [1 0 0], [0]
[0]
[1])


In [86]:
col23switch = Matrix([[1,0,0],[0,0,1],[0,1,0]]);
row23switch = Matrix([[1,1,-1],[0,0,1],[0,1,0]]);
testmask = {'0': col23switch, '1':col23switch};
newtestmad = multiplyPFA(testmad,testmask);
print(newtestmad)

({'0': [1/2   0 1/2]
[  0   0   1]
[  0   1   0], '1': [1/2 1/2   0]
[  0   0   1]
[  0   1   0]}, [1 0 0], [0]
[0]
[1])


In [151]:
for entry in sortproblist(list_probs_witness(testmad,lengths=2)):
    print(entry,homo_image(testhom,entry[0]))

((1, 1), 3/4) (1, 1)
((1, 0), 1/2) (1, 0)
((0, 1), 1/4) (0, 1)
((0, 0), 0) (0, 0)


In [155]:
for entry in sortproblist(list_probs_witness(newtestmad,lengths=range(3))):
    print(entry,entry[1].n().str(base=2))

((1, 0), 3/4) 0.11000000000000000000000000000000000000000000000000000
((0,), 1/2) 0.10000000000000000000000000000000000000000000000000000
((1, 1), 1/2) 0.10000000000000000000000000000000000000000000000000000
((0, 0), 1/4) 0.010000000000000000000000000000000000000000000000000000
((), 0) 0.0000000000000000000000000000000000000000000000000000
((1,), 0) 0.0000000000000000000000000000000000000000000000000000
((0, 1), 0) 0.0000000000000000000000000000000000000000000000000000


In [163]:
accprob_witness(newtestmad,(0,0,0,0,0,0,1)).n().str(base=2)

'0.10101000000000000000000000000000000000000000000000000'

In [13]:
gfa1 = bruteforcevecs(maketuple(([1,0],[0,1],[1/2,0],[0,1/2],[1/2,1/2],[0,0])))

In [14]:
len(gfa1)

1296

In [62]:
f0(x) = 1/2 + (1/6)*x
f1(x) = 1/4 + (1/3)*x

In [2]:
def composelist(funclist):
    l = len(funclist);
    assert l >= 2;
    outfun = funclist[l-1];
    for n in range(1,l):
        outfun = compose(funclist[l-n-1],outfun);
    newf(x) = outfun; #I have no idea why this is necessary, Sage is SO clunky
    return newf;

In [63]:
composelist([f0,f0,f1])(1)

259/432

In [143]:
twit = SW["00001000"][0]; twit

(
{'0': [1/2   0 1/2]                            
[  0   0   1]                                  
[  0   1   0], '1': [  1   0   0]           [0]
[1/2 1/2   0]                               [0]
[1/2 1/2   0]}                   , [1 0 0], [1]
)

In [149]:
twitlist = sortproblist(list_probs_witness(twit,lengths=range(3,18)))

In [150]:
highest_prob(twitlist,lengths="all")

[('000', 5/8),
 ('010', 5/8),
 ('0010', 11/16),
 ('00010', 23/32),
 ('01000', 23/32),
 ('001000', 49/64),
 ('0001000', 101/128),
 ('00001000', 205/256),
 ('000001000', 413/512),
 ('000100000', 413/512),
 ('0000100000', 837/1024),
 ('00000100000', 1685/2048),
 ('000000100000', 3381/4096),
 ('0000000100000', 6773/8192),
 ('0000010000000', 6773/8192),
 ('00000010000000', 13589/16384),
 ('000000010000000', 27221/32768),
 ('0000000010000000', 54485/65536),
 ('00000000010000000', 109013/131072),
 ('00000001000000000', 109013/131072)]

In [6]:
SW["01000"]

[(
{'0': [1/2 1/2   0]                            
[  0   0   1]                                  
[  0   1   0], '1': [  0 1/2 1/2]           [0]
[1/2 1/2   0]                               [0]
[  1   0   0]}                   , [1 0 0], [1]
)]

In [7]:
twit2 = SW["01000"][0]
twitlist2 = sortproblist(list_probs_witness(twit2,lengths=range(3,16)))

In [8]:
highest_prob(twitlist2,lengths="all")

[('010', 1/2),
 ('100', 1/2),
 ('0000', 5/8),
 ('01000', 9/16),
 ('000000', 21/32),
 ('0100000', 37/64),
 ('00000000', 85/128),
 ('010000000', 149/256),
 ('0000000000', 341/512),
 ('01000000000', 597/1024),
 ('000000000000', 1365/2048),
 ('0100000000000', 2389/4096),
 ('00000000000000', 5461/8192),
 ('010000000000000', 9557/16384)]

In [39]:
SW["01001010"]

[(
{'0': [  0   0   1]                            
[1/2   0 1/2]                                  
[  0 1/2 1/2], '1': [1/2   0 1/2]           [0]
[  0   1   0]                               [0]
[1/2 1/2   0]}                   , [1 0 0], [1]
)]

In [11]:
twit3 = SW["01001010"][0]
twitlist3 = sortproblist(list_probs_witness(twit3,lengths=range(3,16)))
s=[print(*p) for p in highest_prob(twitlist3,lengths="all")]

010 3/4
110 3/4
1010 11/16
1110 11/16
01010 3/4
001010 3/4
101010 3/4
1001010 95/128
1101010 95/128
01001010 3/4
001001010 3/4
101001010 3/4
1001001010 767/1024
1101001010 767/1024
01001001010 3/4
001001001010 3/4
101001001010 3/4
1001001001010 6143/8192
1101001001010 6143/8192
01001001001010 3/4
001001001001010 3/4
101001001001010 3/4


In [42]:
SW["00001010"]

[(
{'0': [1/6 5/6]                      
[5/6 1/6], '1': [1/3 2/3]         [1]
[1/2 1/2]}               , [1 0], [0]
),
 (
{'0': [1/6 5/6]                      
[2/3 1/3], '1': [1/3 2/3]         [1]
[1/2 1/2]}               , [1 0], [0]
),
 (
{'0': [  0   1]                      
[5/6 1/6], '1': [1/6 5/6]         [1]
[1/3 2/3]}               , [1 0], [0]
),
 (
{'0': [  0   1]                      
[3/5 2/5], '1': [1/5 4/5]         [1]
[2/5 3/5]}               , [1 0], [0]
),
 (
{'0': [1/6 5/6]                      
[2/3 1/3], '1': [  0   1]         [0]
[5/6 1/6]}               , [1 0], [1]
)]

In [3]:
twit4 = SW["00001010"][4]
twitlist4 = sortproblist(list_probs_witness(twit4,lengths=range(3,16)))
s=[print(*p) for p in highest_prob(twitlist4,lengths="all")]

111 31/36
0000 25/48
11111 991/1296
000010 317/576
1111111 32551/46656
00001010 3889/6912
111111111 1093711/1679616
0000101010 47093/82944
11111111111 37420471/60466176
000010101010 567241/995328
1111111111111 1298308831/2176782336
00001010101010 6817517/11943936
111111111111110 15198673499/26121388032


In [4]:
twit5=SW["00000010"][0];
twitlist5 = sortproblist(list_probs_witness(twit5,lengths=range(3,16)))
s=[print(*p) for p in highest_prob(twitlist5,lengths="all")]

110 10/27
0000 14/27
11010 101/243
000000 110/243
1101010 931/2187
00000010 949/2187
110101010 8423/19683
0000001010 8459/19683
11010101010 75895/177147
000000101010 75967/177147
1101010101010 683231/1594323
00000010101010 683375/1594323
110101010101010 6149431/14348907


In [5]:
twit5

(
{'0': [  0   1]                      
[2/3 1/3], '1': [1/6 5/6]         [1]
[1/2 1/2]}               , [1 0], [0]
)

In [12]:
twit6=SW["0000010"][0]
twitlist6 = sortproblist(list_probs_witness(twit6,lengths=range(3,16)))
s=[print(*p) for p in highest_prob(twitlist6,lengths="all")]

000 399/512
1010 595/1024
00000 23135/32768
101010 41167/65536
0000010 1403339/2097152
10101010 2699515/4194304
000001010 88190375/134217728
1010101010 174130327/268435456
00000101010 5610094259/8589934592
101010101010 11172929635/17179869184
0000010101010 358330148015/549755813888
10101010101010 715667859487/1099511627776
000001010101010 22918095897179/35184372088832


In [13]:
twit6

(
{'0': [1/8 7/8]                      
[  1   0], '1': [1/2 1/2]         [0]
[7/8 1/8]}               , [1 0], [1]
)